In [1]:
import os
import torch
from d2l import torch as d2l

英语-法语的句子对所组成的数据集

下载数据集，读取几条看一下
此时的第一行：Go.	Va !
         123 4 5678
         含有八个字符
         则：juzi=raw_text[:8]，就是Go.	Va !

In [2]:
#@save
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip',
                           '94646ad1522d915e7b0f9296181140edcf86a4f5')

#@save
def read_data_nmt():
    """载入“英语－法语”数据集"""
    data_dir = d2l.download_extract('fra-eng')
    with open(os.path.join(data_dir, 'fra.txt'), 'r',
             encoding='utf-8') as f:
        return f.read()

raw_text = read_data_nmt()
print(type(raw_text),'----')
juzi=raw_text[:75]
print(juzi,'-----------',len(juzi))
juzi=raw_text[:8]
print(juzi,'-----------',len(juzi))

<class 'str'> ----
Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !
 ----------- 75
Go.	Va ! ----------- 8


原始文本数据需要经过几个预处理步骤。

例如，我们用空格代替不间断空格（non-breaking space）， 
使用小写字母替换大写字母，并在单词和标点符号之间插入空格。

In [3]:
#@save
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


In [4]:
juzi = preprocess_nmt(juzi)
print(juzi)

go .	va !


词元化

In [5]:
# enumerate（）函数，参考菜鸟教程
# 将一个可迭代的序列，每一个元素的前面配备上一个index，默认start=0
seq = ['one', 'two', 'three']
print(seq)
for index,element in enumerate(seq,start=10):
    print(index,element)

['one', 'two', 'three']
10 one
11 two
12 three


In [6]:
#@save
def tokenize_nmt(text, num_examples=None):
    """词元化“英语－法语”数据数据集"""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
#         if i==0:
#             print(i)
#             print(line)
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
#         print(type(parts),'---',parts[0],'---',parts[1],'---',parts)
        #break
        if len(parts) == 2:
            source.append(parts[0].split(' '))
#             print(source)
            target.append(parts[1].split(' '))
#             print(target)
#             break
    return source, target

source, target = tokenize_nmt(text)
source[:6], target[:6]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!']])

In [8]:
src_vocab = d2l.Vocab(source, min_freq=2,
                      reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab),type(src_vocab),

(10012, d2l.torch.Vocab)